In [1]:
!pip install -r requirements.txt

  Using cached llama_parse-0.4.4-py3-none-any.whl (8.0 kB)
  Using cached llama_index_embeddings_fastembed-0.1.4-py3-none-any.whl (2.7 kB)
  Using cached fastembed-0.2.7-py3-none-any.whl (27 kB)
  Using cached llama_index_vector_stores_qdrant-0.2.8-py3-none-any.whl (9.5 kB)
  Using cached qdrant_client-1.9.1-py3-none-any.whl (229 kB)
  Using cached llama_index_embeddings_ollama-0.1.2-py3-none-any.whl (2.8 kB)
  Using cached llama_index_llms_ollama-0.1.5-py3-none-any.whl (3.6 kB)
  Using cached llama_index_llms_groq-0.1.4-py3-none-any.whl (2.9 kB)
     ---------------------------------------- 15.4/15.4 MB 5.8 MB/s eta 0:00:00
  Using cached llama_index_indices_managed_llama_cloud-0.1.6-py3-none-any.whl (6.7 kB)
  Using cached llama_index_embeddings_openai-0.1.10-py3-none-any.whl (6.2 kB)
  Using cached llama_index_multi_modal_llms_openai-0.1.6-py3-none-any.whl (5.8 kB)
  Using cached llama_index_program_openai-0.1.6-py3-none-any.whl (5.2 kB)
  Using cached llama_index_question_gen_opena

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.41 requires requests_mock, which is not installed.
tensorflow-intel 2.15.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.27.0 which is incompatible.
tensorboard 2.15.1 requires protobuf<4.24,>=3.19.6, but you have protobuf 5.27.0 which is incompatible.
sqlmodel 0.0.8 requires SQLAlchemy<=1.4.41,>=1.4.17, but you have sqlalchemy 2.0.30 which is incompatible.
opentelemetry-proto 1.24.0 requires protobuf<5.0,>=3.19, but you have protobuf 5.27.0 which is incompatible.
googleapis-common-protos 1.63.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0.dev0,>=3.19.5, but you have protobuf 5.27.0 which is incompatible.
fastapi-utils 0.2.1 requires sqlalchemy<2.0.0,>=1.3.12, but you have sqlalchemy 2.

In [2]:
import os
import nest_asyncio
nest_asyncio.apply()

In [3]:
from getpass import getpass
LLAMAPARSE_API_KEY = getpass()

········


In [4]:
os.environ['LLAMAPARSE_API_KEY'] = LLAMAPARSE_API_KEY

In [5]:
from llama_parse import LlamaParse

In [6]:
llama_parse_documents = LlamaParse(api_key=LLAMAPARSE_API_KEY, result_type="markdown").load_data("Practice of Brahmacharya - Swami Sivananda.pdf")

Started parsing the file under job_id 3bab48f8-b100-4333-bc22-8fbff50d90fd
.

In [7]:
len(llama_parse_documents)

1

In [8]:
llama_parse_documents[0].text[:200]

"# PRACTICE OF BRAHMACHARYA\n\nBy\n\nSRI SWAMI SIVANANDA\n\n6(59(\x0f,/29(\x0f,*,9(\x0f\n\n385,)<\x0f,0(',7$7(\x0f\n\n5($/,=(\n\nSri Swami Sivananda\n\nFounder of\n\nThe Divine Life Society\n\nA DIVINE LIFE SOCIETY PUBLICATION\n---\nFir"

In [9]:
type(llama_parse_documents)

list

In [10]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import VectorStoreIndex, StorageContext
import qdrant_client

In [11]:
QDRANT_URL = getpass()

········


In [12]:
os.environ['QDRANT_URL'] = QDRANT_URL

In [13]:
QDRANT_API_KEY = getpass()

········


In [14]:
os.environ['QDRANT_API_KEY'] = QDRANT_API_KEY

In [15]:
import pickle
# Defining a function to load parsed data if available, or parse if not
def load_or_parse_data():
    data_file = "./data/parsed_data.pkl"

    if os.path.exists(data_file):
        # Loading the parsed data from the file
        with open(data_file, "rb") as f:
            parsed_data = pickle.load(f)
    else:
        # Performing the parsing step and store the result in llama_parse_documents
        # llama_parse_documents = LlamaParse(api_key=llamaparse_api_key, result_type="markdown").load_data("./data/presentation.pptx")
        llama_parse_documents = LlamaParse(api_key=LLAMAPARSE_API_KEY, result_type="markdown").load_data(["Practice of Brahmacharya - Swami Sivananda.pdf"])

        # Saving the parsed data to a file
        with open(data_file, "wb") as f:
            pickle.dump(llama_parse_documents, f)

        # Setting the parsed data to the variable
        parsed_data = llama_parse_documents

    return parsed_data

In [16]:
######### FastEmbedEmbeddings #############

# by default llamaindex uses OpenAI models
from llama_index.embeddings.fastembed import FastEmbedEmbedding
embed_model = FastEmbedEmbedding(model_name="BAAI/bge-base-en-v1.5")

""" embed_model = OllamaEmbedding(
    model_name="nomic-embed-text",
    #model_name="llama2",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0},
) """

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

D:\Anaconda\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hp\AppData\Local\Temp\fastembed_cache\models--qdrant--bge-base-en-v1.5-onnx-q. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


' embed_model = OllamaEmbedding(\n    model_name="nomic-embed-text",\n    #model_name="llama2",\n    base_url="http://localhost:11434",\n    ollama_additional_kwargs={"mirostat": 0},\n) '

In [17]:
#### Setting embed_model other than openAI ( by default used openAI's model)
from llama_index.core import Settings

Settings.embed_model = embed_model

In [18]:
######### Groq API ###########
from llama_index.llms.groq import Groq

In [19]:
GROQ_API_KEY = getpass()

········


In [20]:
os.environ['GROQ_API_KEY'] = GROQ_API_KEY

In [21]:
llm = Groq(model="mixtral-8x7b-32768", api_key=GROQ_API_KEY)

In [22]:
Settings.llm = llm

client = qdrant_client.QdrantClient(api_key=QDRANT_API_KEY, url=QDRANT_URL,)

In [23]:
vector_store = QdrantVectorStore(client=client, collection_name='qdrant_rag')
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(documents=llama_parse_documents, storage_context=storage_context, show_progress=True)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/131 [00:00<?, ?it/s]

In [24]:
index.storage_context.persist()

In [26]:
# creating a query engine for the index
query_engine = index.as_query_engine()
query = "What are the ideologies of Swami Sivananda about Brahmacharya? Answer in detail make it simple to read"

# querying the query engine
response = query_engine.query(query)
print(response)

Swami Sivananda emphasizes the importance of brahmacharya, which is often translated as celibacy or sexual continence, as a fundamental principle for self-realization and divine perfection. He teaches that brahmacharya is the basis of a radiant and enthusiastic divine life, and it is an essential aspect of the gospel of Gurudev's teachings for mankind.

Swami Sivananda explains that the aspiration for self-realization and atma jnana (self-knowledge) leads to a principled life, which in turn fosters self-control, self-mastery, and character. This character forms the foundation for brahmacharya, which is the basis for immortality, divine perfection, liberation, and a fulfilling life.

He highlights the significance of brahmacharya by pointing out that the present-day degradation and moral decline can be attributed to the flouting of scriptural norms and the lack of understanding of the ancient wisdom regarding sex and its regulation. Swami Sivananda aims to fill this lacuna by offering g